In [1]:
# Install Java, Spark, and Findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.6/spark-2.4.6-bin-hadoop2.7.tgz
!tar xf spark-2.4.6-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.6-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETLProjectAnalysis").getOrCreate()

In [73]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Mobile_Electronics_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Mobile_Electronics_v1_00.tsv.gz"), header=True, sep="\t")
df.show(truncate=False)

+-----------+-----------+--------------+----------+--------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------+-----------+-------------+-----------+----+-----------------+-------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [74]:
print(f"Total Number rows in the dataframe are {df.count()}")

Total Number rows in the dataframe are 104975


In [75]:
review_id_table=df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_table.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| R8MEA6IGAHO0B|   20422322|B00MC4CED8|     217304173| 2015-08-31|
|R31LOQ8JGLPRLK|   40835037|B00OQMFG1Q|     137313254| 2015-08-31|
|R2Y0MM9YE6OP3P|   51469641|B00QERR5CY|      82850235| 2015-08-31|
| RRB9C05HDOD4O|    4332923|B00QUFTPV4|     221169481| 2015-08-31|
|R26I2RI1GFV8QG|   44855305|B0067XVNTG|     563475445| 2015-08-31|
| RY8DDL22YG4R5|    7846966|B00KA6CCVY|     713636156| 2015-08-31|
|R2AT2426ZHFUHH|   21299354|B00MJCDPM2|     754171872| 2015-08-31|
|R3RRXU2R23NMQ9|   28902968|B00ET5AWBY|     508373185| 2015-08-31|
|R250PR8VJUZ62F|    2074438|B00Z9K379C|     759387370| 2015-08-31|
| RBEMQ29WJBHYG|   48701722|B00YO3UYXW|     913911909| 2015-08-31|
|R19VVIUT4BZCMT|    2990247|B011W26BU2|     938399424| 2015-08-31|
|R1DT8JJUQHUKUL|   52946117|B00H8PI78C|     661515294| 2015-08

In [76]:
review_id_table.dtypes

[('review_id', 'string'),
 ('customer_id', 'string'),
 ('product_id', 'string'),
 ('product_parent', 'string'),
 ('review_date', 'string')]

In [77]:
from pyspark.sql.types import IntegerType, DateType
review_id_table=review_id_table.withColumn("customer_id", review_id_table["customer_id"].cast(IntegerType()))
review_id_table=review_id_table.withColumn("product_id", review_id_table["product_id"].cast(IntegerType()))
review_id_table=review_id_table.withColumn("review_date", review_id_table["review_date"].cast(DateType()))


In [78]:
products=df.select(["product_id", "product_title" ])
products=products.dropDuplicates()
products.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B01075GRAO|ANYSECU QYT KT890...|
|B00QVXNV60|BRG AH2000G-ANR A...|
|B004ECO5VA|Holux Mini Smart ...|
|B00LGNKLXG|Aweek® Car Antenn...|
|B00M1Y87I0|      sentey ls 4063|
|B002ABX29I|Micca Slim Portab...|
|B00XXA2T2G|Centurion GT1 Car...|
|B00Q5A5PD6|HYUNDAI HCR-580 V...|
|B00VFEK8IY|Z ZTDM Mini Stere...|
|B00L85M8VU|Settonbrothers Ul...|
|B00SUXD5GU|[NEW RELEASE] JEB...|
|B00KJ7MVYW|Fincibo (TM) Appl...|
|B00A6YQQS0|Minisuit Aero Fol...|
|B009S99IYI|PIO TSG1644R 6.5 ...|
|B006PAZCXY|eForCity® Anti-Gl...|
|B00OXD61OE|SHE'O® 2x Cool No...|
|B000IBMI1K|RCA 200-Watt Mono...|
|B00UN6955M|Aisster(TM) Lot 1...|
|B00V684NCQ|LSD 8GB Flash Dri...|
|B00A609WOO|eForCity Hybrid S...|
+----------+--------------------+
only showing top 20 rows



In [79]:
products.count()

25801

In [80]:
from pyspark.sql import functions as F
customers=df.groupby("customer_id").agg( F.count("customer_id"))
customers=customers.withColumnRenamed("count(customer_id)", "customer_count")
customers.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|    3962209|             1|
|   24644096|             1|
|   24659376|             1|
|   24486730|             2|
|   36657077|             1|
|   11418933|             1|
|   32573306|             1|
|   20754832|             1|
|   14658656|             1|
|   18419577|             1|
|   28239403|             1|
|    2284327|             1|
|   10631491|             1|
|   22438646|             1|
|    4365384|             1|
|   18635888|             1|
|   33820066|             1|
|    1942116|             1|
|   28785561|             1|
|   15549256|             1|
+-----------+--------------+
only showing top 20 rows



In [81]:
vine_table=df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_table.dtypes

[('review_id', 'string'),
 ('star_rating', 'string'),
 ('helpful_votes', 'string'),
 ('total_votes', 'string'),
 ('vine', 'string')]

In [82]:
vine_table=vine_table.withColumn("star_rating", vine_table["star_rating"].cast(IntegerType()))
vine_table=vine_table.withColumn("helpful_votes", vine_table["helpful_votes"].cast(IntegerType()))
vine_table=vine_table.withColumn("total_votes", vine_table["total_votes"].cast(IntegerType()))
vine_table.show()


+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| R8MEA6IGAHO0B|          5|            0|          0|   N|
|R31LOQ8JGLPRLK|          5|            0|          1|   N|
|R2Y0MM9YE6OP3P|          5|            0|          0|   N|
| RRB9C05HDOD4O|          4|            0|          0|   N|
|R26I2RI1GFV8QG|          2|            0|          0|   N|
| RY8DDL22YG4R5|          3|            0|          1|   N|
|R2AT2426ZHFUHH|          3|            0|          1|   N|
|R3RRXU2R23NMQ9|          5|            0|          0|   N|
|R250PR8VJUZ62F|          4|            0|          2|   N|
| RBEMQ29WJBHYG|          5|          164|        168|   N|
|R19VVIUT4BZCMT|          5|            2|          2|   N|
|R1DT8JJUQHUKUL|          3|            0|          1|   N|
| RLIL6S9OGM3YA|          5|            0|          0|   N|
|R34EZZ68VYPHO0|          1|            

In [83]:
# Configure settings for RDS
mode = "append"
# Enter credentials here
jdbc_url="jdbc:postgresql://<RDS-Endpoint>:5432/amazon-sentiments"
config = {"user":"root", 
          "password": "<RDS password>", 
          "driver":"org.postgresql.Driver"}


In [14]:
review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [15]:
products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [16]:
customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [17]:
vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

In [84]:
from pyspark.sql.functions import col,when,count, sum, round
from pyspark.sql.window import Window
star_rating=df.groupby("star_rating").agg(count(when(col("vine") == "N", True)), count(when(col("vine") == "Y", True)))
star_rating=star_rating.withColumnRenamed("count(CASE WHEN (vine = N) THEN true END)", "non-vine").withColumnRenamed("count(CASE WHEN (vine = Y) THEN true END)", "vine")\
.sort(col("star_rating").desc())\
.withColumn('percent-non-vine', col('non-vine')/sum('non-vine').over(Window.partitionBy()))\
.withColumn('percent-vine', col('vine')/sum('vine').over(Window.partitionBy()))\
.show()

+-----------+--------+----+-------------------+------------------+
|star_rating|non-vine|vine|   percent-non-vine|      percent-vine|
+-----------+--------+----+-------------------+------------------+
|          5|   52249|   6| 0.4978133902455291|0.3333333333333333|
|          4|   18082|   6|0.17228007660279923|0.3333333333333333|
|          3|    9730|   4|0.09270463142048649|0.2222222222222222|
|          2|    7309|   2|0.06963804224587213|0.1111111111111111|
|          1|   17587|   0|0.16756385948531302|               0.0|
+-----------+--------+----+-------------------+------------------+



In [85]:
helpful_vote=df.groupby("vine").agg(count("vine"), round(F.avg("star_rating"),2), round(F.sum("helpful_votes")/count("vine"), 2), round(F.sum("total_votes")/count("vine"), 2))\
.withColumnRenamed("round((sum(helpful_votes) / count(vine)), 2)", "Avg Helpful Votes")\
.withColumnRenamed("round(avg(star_rating), 2)", "Avg Star Rating")\
.withColumnRenamed("round((sum(total_votes) / count(vine)), 2)", "Avg Total Votes").show()


+----+-----------+---------------+-----------------+---------------+
|vine|count(vine)|Avg Star Rating|Avg Helpful Votes|Avg Total Votes|
+----+-----------+---------------+-----------------+---------------+
|   Y|         18|           3.89|            41.89|           50.5|
|   N|     104957|           3.76|             1.24|           1.61|
+----+-----------+---------------+-----------------+---------------+



In [94]:
advertised_product=df.groupby("product_title").agg(count(when(col("vine") == "N", True)), count(when(col("vine") == "Y", True)))\
.sort(col("count(CASE WHEN (vine = Y) THEN true END)").desc()).show()

+--------------------+-----------------------------------------+-----------------------------------------+
|       product_title|count(CASE WHEN (vine = N) THEN true END)|count(CASE WHEN (vine = Y) THEN true END)|
+--------------------+-----------------------------------------+-----------------------------------------+
|Orbotix Sphero 2....|                                       25|                                       10|
|Avid Vocal Studio...|                                       33|                                        8|
|Apple iPad Air Ca...|                                       13|                                        0|
|100% Waterproof A...|                                       10|                                        0|
|Saitek AI FMK578 ...|                                        2|                                        0|
|Generic Car Radar...|                                        1|                                        0|
|Mivizu Universal ...|               

In [89]:
# Samsung Counter Top Microwave
ndf=df.filter(df.product_title =="Orbotix Sphero 2.0 App Controlled Robotic Ball - Retail Packaging - White/Blue")
Product_bias=ndf.groupby("vine").agg(count("vine"), round(F.avg("star_rating"),2), round(F.sum("helpful_votes")/count("vine"), 2), round(F.sum("total_votes")/count("vine"), 2))\
.withColumnRenamed("round((sum(helpful_votes) / count(vine)), 2)", "Avg Helpful Votes")\
.withColumnRenamed("round(avg(star_rating), 2)", "Avg Star Rating")\
.withColumnRenamed("round((sum(total_votes) / count(vine)), 2)", "Avg Total Votes").show()


+----+-----------+---------------+-----------------+---------------+
|vine|count(vine)|Avg Star Rating|Avg Helpful Votes|Avg Total Votes|
+----+-----------+---------------+-----------------+---------------+
|   Y|         10|            4.2|             74.0|           88.4|
|   N|         25|           3.84|             6.36|          12.28|
+----+-----------+---------------+-----------------+---------------+



In [91]:
ndf=df.filter(df.product_title =="Orbotix Sphero 2.0 App Controlled Robotic Ball - Retail Packaging - White/Blue")
product_star_rating=ndf.groupby("star_rating").agg(count(when(col("vine") == "N", True)), count(when(col("vine") == "Y", True)))
product_star_rating=product_star_rating.withColumnRenamed("count(CASE WHEN (vine = N) THEN true END)", "non-vine").withColumnRenamed("count(CASE WHEN (vine = Y) THEN true END)", "vine")\
.sort(col("star_rating").desc())\
.withColumn('percent-non-vine', col('non-vine')/sum('non-vine').over(Window.partitionBy()))\
.withColumn('percent-vine', col('vine')/sum('vine').over(Window.partitionBy()))\
.show()

+-----------+--------+----+----------------+------------+
|star_rating|non-vine|vine|percent-non-vine|percent-vine|
+-----------+--------+----+----------------+------------+
|          5|      15|   4|             0.6|         0.4|
|          4|       1|   4|            0.04|         0.4|
|          3|       2|   2|            0.08|         0.2|
|          2|       4|   0|            0.16|         0.0|
|          1|       3|   0|            0.12|         0.0|
+-----------+--------+----+----------------+------------+



In [93]:
ndf=df.filter(df.product_title =="Orbotix Sphero 2.0 App Controlled Robotic Ball - Retail Packaging - White/Blue")
customer_ids=ndf.groupby("customer_id").agg(count(when(col("vine") == "N", True)), count(when(col("vine") == "Y", True))).sort(col("count(CASE WHEN (vine = Y) THEN true END)").desc()).show()

+-----------+-----------------------------------------+-----------------------------------------+
|customer_id|count(CASE WHEN (vine = N) THEN true END)|count(CASE WHEN (vine = Y) THEN true END)|
+-----------+-----------------------------------------+-----------------------------------------+
|   52923654|                                        0|                                        1|
|   52433849|                                        0|                                        1|
|   52685392|                                        0|                                        1|
|   48814959|                                        0|                                        1|
|   44728245|                                        0|                                        1|
|   14220975|                                        0|                                        1|
|   34641748|                                        0|                                        1|
|   52477683|       